In [1]:
import pandas as pd
import requests
import json
import time
import random
import os
from vpn import VPNChanger
import gc
import asyncio
import aiohttp
from typing import Dict

In [ ]:
def carregar_tabela_referencia():
    url = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarTabelaDeReferencia'
    
    try:
        response = requests.post(url, timeout=6)
        response.raise_for_status()
    # Retorne os dados de resposta como um objeto JSON
        return response.json()

    except requests.exceptions.HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except requests.exceptions.ConnectionError as conn_err:
        print(f'Error connecting: {conn_err}')
    except requests.exceptions.Timeout as timeout_err:
        print(f'Timeout error: {timeout_err}')
    except requests.exceptions.RequestException as req_err:
        print(f'Error: {req_err}')

In [ ]:
tb_ref_data = carregar_tabela_referencia()
tb_ref_data = pd.DataFrame(tb_ref_data)

In [ ]:
codVeiculo = {
    'cod_tipo_veiculo': [1,2,3],
    'veiculo': ['carro', 'moto', 'caminhao']
}

codVeiculo = pd.DataFrame(codVeiculo)

In [ ]:
def consultar_marcas(tbl_ref, tbl_veiculo):
    url = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarMarcas'
    headers = {'Content-Type': 'application/json'}
    data = {
        'codigoTabelaReferencia': tbl_ref,  # ou outro valor que você obteve da função carregarTabelaReferencia
        'codigoTipoVeiculo': tbl_veiculo,  # o código correspondente ao tipo do veículo
    }
    
    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        return response.json()

    except requests.exceptions.HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except requests.exceptions.ConnectionError as conn_err:
        print(f'Error connecting: {conn_err}')
    except requests.exceptions.Timeout as timeout_err:
        print(f'Timeout error: {timeout_err}')
    except requests.exceptions.RequestException as req_err:
        print(f'Error: {req_err}')

In [ ]:
marcas_veiculos = []
for i in range(len(codVeiculo)):
    time.sleep(random.randint(1,3))
    marcas = consultar_marcas(str(tb_ref_data['Codigo'].max()), str(i+1))
    for marca in marcas: 
        marca['cod_veiculo'] = i+1
    marcas_veiculos.extend(marcas)

df_marcas = pd.DataFrame(marcas_veiculos)

In [ ]:
df_marcas.to_csv('marcas.csv', sep=';', index=False)
df_marcas

In [ ]:
df_marcas = pd.read_csv('marcas.csv', sep=';')
df_marcas

In [ ]:
novas_linhas = []

df_conferenciaModelos = pd.DataFrame()
for index, row in df_marcas.iterrows():
    time.sleep(random.randint(1,3))
    parametros = {
        'codigoTipoVeiculo': str(row['cod_veiculo']),
        'codigoTabelaReferencia': str(tb_ref_data['Codigo'].max()),
        'codigoModelo': '',
        'codigoMarca': row['Value'],
        'ano': '',
        'codigoTipoCombustivel': '',
        'anoModelo': '',
        'modeloCodigoExterno': ''
    }

    headers = {'Content-Type': 'application/json'}
    urlConsulta = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarModelos'
    response = requests.post(urlConsulta, headers=headers, data=json.dumps(parametros))
    modelos = response.json()
    
    for modelo in modelos['Modelos']:
        nova_linha = row.copy()
        nova_linha['Modelo'] = modelo['Label']
        nova_linha['codigoModelo'] = modelo['Value']
        novas_linhas.append(nova_linha)

    df_conferenciaModelos = pd.DataFrame(novas_linhas)
    df_conferenciaModelos.to_csv('conferenciaModelos.csv', sep=';', index=False)

df_marcas_modelos = pd.DataFrame(novas_linhas)
df_marcas_modelos.to_csv('marcas_modelos.csv', sep = ';', index=False)


In [ ]:
df_marcas_modelos = pd.read_csv('marcas_modelos.csv', sep=';')
df_marcas_modelos

In [2]:
def get_current_index(control):
    control_file = f"{control}.txt"
    if os.path.exists(control_file):
        with open(control_file, "r") as file:
            return int(file.read().strip())
    return 0

def update_current_index(index, control):
    control_file = f"{control}.txt"
    with open(control_file, "w") as file:
        file.write(str(index))

In [ ]:
vpn_changer = VPNChanger()
vpn_files = vpn_changer.get_vpn_files()

vpn_changer.connect_vpn(vpn_files[random.randint(0, len(vpn_files) - 1)])

if os.path.exists('conferenciaModelosAnos.csv'):
    df_conferenciaModelosAnos = pd.read_csv('conferenciaModelosAnos.csv')
    novas_linhas = df_conferenciaModelosAnos.to_dict('records')
else:
    df_conferenciaModelosAnos = pd.DataFrame()
    novas_linhas = []

df_conferenciaModelosAnos = pd.DataFrame()
start_index = get_current_index('control_AnoModelo')
for index, row in df_marcas_modelos.iloc[start_index:].iterrows():
    success = False
    while not success:
        try:
            if index % 300 == 0:
                vpn_changer.change_ip()

            parametros = {
                'codigoTipoVeiculo': str(row['cod_veiculo']),
                'codigoTabelaReferencia': str(tb_ref_data['Codigo'].max()),
                'codigoModelo': str(row['codigoModelo']),
                'codigoMarca': str(row['Value']),
            }

            urlConsulta = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarAnoModelo'
            headers = {'Content-Type': 'application/json'}
            response = requests.post(urlConsulta, headers=headers, data=json.dumps(parametros))
            anos = response.json()
            
            for ano in anos:
                nova_linha = row.copy()
                nova_linha['Ano'] = ano['Label']
                nova_linha['codigoAno'] = ano['Value']
                novas_linhas.append(nova_linha)

            df_conferenciaModelosAnos = pd.DataFrame(novas_linhas)
            df_conferenciaModelosAnos.to_csv('conferenciaModelosAnos.csv', index=False)
            update_current_index(index + 1, "control_AnoModelo")

            success = True
        except:
            print(f"Erro ao processar o registro {index}")
            vpn_changer.change_ip()
df_marcas_modelos_anos = pd.DataFrame(novas_linhas)
df_marcas_modelos_anos.to_csv('marcas_modelos_anos.csv', sep = ';', index=False)

vpn_changer.disconnect_vpn()

In [ ]:
df_marcas_modelos_anos = pd.read_csv('marcas_modelos_anos.csv', sep=';')
df_marcas_modelos_anos = df_marcas_modelos_anos.rename(columns={'Label':'Marca','Value':'cod_marca'})
df_marcas_modelos_anos

In [ ]:
linha = 12318

parametros = {
    "codigoTabelaReferencia": int(tb_ref_data['Codigo'].max()),
    "codigoMarca": int(df_marcas_modelos_anos['cod_marca'].iloc[linha]),
    "codigoTipoVeiculo": int(df_marcas_modelos_anos['cod_veiculo'].iloc[linha]),
    "codigoModelo": int(df_marcas_modelos_anos['codigoModelo'].iloc[linha]),
    "anoModelo": int(df_marcas_modelos_anos['codigoAno'].iloc[linha].split('-')[0]),
    "codigoTipoCombustivel": int(df_marcas_modelos_anos['codigoAno'].iloc[linha].split('-')[1]),
    "ano": str(df_marcas_modelos_anos['codigoAno'].iloc[linha]),
    "tipoConsulta": "tradicional"
}

urlConsulta = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarValorComTodosParametros'
headers = {'Content-Type': 'application/json'}
response = requests.post(urlConsulta, headers=headers, data=json.dumps(parametros))
preco = response.json()
preco

In [ ]:
# vpn_changer = VPNChanger()
# vpn_files = vpn_changer.get_vpn_files()

# vpn_changer.connect_vpn(vpn_files[random.randint(0, len(vpn_files) - 1)])

if os.path.exists('conferenciaModelosAnosPreco.csv'):
    df_conferenciaModelosAnosPreco = pd.read_csv('conferenciaModelosAnosPreco.csv')
    nova_tabela_precos = df_conferenciaModelosAnosPreco.to_dict('records')
else:
    df_conferenciaModelosAnosPreco = pd.DataFrame()
    nova_tabela_precos = []

df_conferenciaModelosAnos = pd.DataFrame()
start_index = get_current_index('control_AnoModeloPreco')
for index, row in df_marcas_modelos_anos.iloc[start_index:].iterrows():
    success = False
    while not success:
        try:
            # if index % 200 == 0 and index > 0:
            #    vpn_changer.change_ip()

            parametros = {
                "codigoTabelaReferencia": int(tb_ref_data['Codigo'].max()),
                "codigoMarca": int(row['cod_marca']),
                "codigoTipoVeiculo": int(row['cod_veiculo']),
                "codigoModelo": int(row['codigoModelo']),
                "anoModelo": int(row['codigoAno'].split('-')[0]),
                "codigoTipoCombustivel": int(row['codigoAno'].split('-')[1]),
                "ano": str(row['codigoAno']),
                "tipoConsulta": "tradicional"
            }

            urlConsulta = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarValorComTodosParametros'
            headers = {'Content-Type': 'application/json'}
            response = requests.post(urlConsulta, headers=headers, data=json.dumps(parametros))
            preco = response.json()

            novo_registro = {
                'Valor': preco['Valor'],
                'Marca': preco['Marca'],
                'Modelo': preco['Modelo'],
                'AnoModelo': preco['AnoModelo'],
                'Combustivel': preco['Combustivel'],
                'CodigoFipe': preco['CodigoFipe'],
                'MesReferencia': preco['MesReferencia'],
                'Autenticacao': preco['Autenticacao'],
                'TipoVeiculo': preco['TipoVeiculo'],
                'SiglaCombustivel': preco['SiglaCombustivel'],
                'DataConsulta': preco['DataConsulta']
            }
            nova_tabela_precos.append(novo_registro)

            df_conferenciaModelosAnosPreco = pd.DataFrame(nova_tabela_precos)
            df_conferenciaModelosAnosPreco.to_csv('conferenciaModelosAnosPreco.csv', index=False)
            update_current_index(index + 1, "control_AnoModeloPreco")

            success = True
        except:
            print(f"Erro ao processar o registro {index}")
            # vpn_changer.change_ip()
    del row
    gc.collect()
df_marcas_modelos_anos_precos = pd.DataFrame(nova_tabela_precos)

# vpn_changer.disconnect_vpn()

In [ ]:
import csv

if not os.path.exists('conferenciaModelosAnosPreco.csv'):
    with open('conferenciaModelosAnosPreco.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['Valor', 'Marca', 'Modelo', 'AnoModelo', 'Combustivel', 'CodigoFipe', 'MesReferencia', 'Autenticacao', 'TipoVeiculo', 'SiglaCombustivel', 'DataConsulta'])

start_index = get_current_index('control_AnoModeloPreco')
for index, row in df_marcas_modelos_anos.iloc[start_index:].iterrows():
    success = False
    while not success:
        try:
            parametros = {
                "codigoTabelaReferencia": int(tb_ref_data['Codigo'].max()),
                "codigoMarca": int(row['cod_marca']),
                "codigoTipoVeiculo": int(row['cod_veiculo']),
                "codigoModelo": int(row['codigoModelo']),
                "anoModelo": int(row['codigoAno'].split('-')[0]),
                "codigoTipoCombustivel": int(row['codigoAno'].split('-')[1]),
                "ano": str(row['codigoAno']),
                "tipoConsulta": "tradicional"
            }

            urlConsulta = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarValorComTodosParametros'
            headers = {'Content-Type': 'application/json'}
            response = requests.post(urlConsulta, headers=headers, data=json.dumps(parametros))
            preco = response.json()

            with open('conferenciaModelosAnosPreco.csv', 'a', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow([preco['Valor'], preco['Marca'], preco['Modelo'], preco['AnoModelo'], preco['Combustivel'], preco['CodigoFipe'], preco['MesReferencia'], preco['Autenticacao'], preco['TipoVeiculo'], preco['SiglaCombustivel'], preco['DataConsulta']])

            update_current_index(index + 1, "control_AnoModeloPreco")

            success = True
        except:
            print(f"Erro ao processar o registro {index}")
            
    del row
    gc.collect()

df_marcas_modelos_anos_precos = pd.read_csv('conferenciaModelosAnosPreco.csv')


In [ ]:
from datetime import datetime

In [ ]:
df_marcas_modelos_anos = pd.read_csv('marcas_modelos_anos.csv', sep=';')
df_marcas_modelos_anos = df_marcas_modelos_anos.rename(columns={'Label':'Marca','Value':'cod_marca'})
df_marcas_modelos_anos['data_hora_atualizacao'] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
df_marcas_modelos_anos['data_hora_atualizacao'] = pd.to_datetime(df_marcas_modelos_anos['data_hora_atualizacao'])
df_marcas_modelos_anos = df_marcas_modelos_anos.replace(to_replace='32000', value='2000', regex=True)
df_marcas_modelos_anos

In [ ]:
df_referencia = tb_ref_data.copy()
meses_pt_para_en = {
    'janeiro': 'January',
    'fevereiro': 'February',
    'março': 'March',
    'abril': 'April',
    'maio': 'May',
    'junho': 'June',
    'julho': 'July',
    'agosto': 'August',
    'setembro': 'September',
    'outubro': 'October',
    'novembro': 'November',
    'dezembro': 'December'
}
df_referencia['Mes'] = df_referencia['Mes'].str.strip()
for mes_pt, mes_en in meses_pt_para_en.items():
    df_referencia['Mes'] = df_referencia['Mes'].str.replace(mes_pt, mes_en)

df_referencia['Mes'] = pd.to_datetime(df_referencia['Mes'], format='%B/%Y')

In [ ]:
df_referencia = df_referencia.sort_values(by='Mes', ascending=False)
df_referencia

In [ ]:
linhas_novas = []


for idx, row in df_marcas_modelos_anos.iterrows():
    ano_modelo = int(row['Ano'].split(' ')[0])


    if ano_modelo < 2001:
        data_inicio = datetime(year=2001, month=1, day=1)
    else:
        data_inicio = datetime(year=ano_modelo-1, month=1, day=1)

    df_ref_filtrado = df_referencia[(df_referencia['Mes'] >= data_inicio) & (df_referencia['Mes'] <= datetime(year=2023, month=7, day=1))]


    for i, ref_row in df_ref_filtrado.iterrows():
        nova_linha = row.copy().to_dict()
        nova_linha['data_hora_atualizacao'] = ref_row['Mes']
        nova_linha['Codigo'] = ref_row['Codigo']
        linhas_novas.append(nova_linha)


df_consulta_total = pd.DataFrame(linhas_novas)

In [ ]:
df_consulta_total.to_csv('consulta_total.csv', index=False)

In [3]:
df_consulta_total = pd.read_csv('consulta_total.csv')

C:\Users\victo\AppData\Local\Temp\ipykernel_6836\2186243507.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_consulta_total = pd.read_csv('consulta_total.csv')


In [ ]:
import csv

if not os.path.exists('conferenciaHistoricoPrecos.csv'):
    with open('conferenciaHistoricoPrecos.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['Valor', 'Marca', 'Modelo', 'AnoModelo', 'Combustivel', 'CodigoFipe', 'MesReferencia', 'Autenticacao', 'TipoVeiculo', 'SiglaCombustivel', 'DataConsulta'])

start_index = get_current_index('control_HistoricoPrecos')
for index, row in df_consulta_total.iloc[start_index:].iterrows():
    success = False
    while not success:
        try:
            parametros = {
                "codigoTabelaReferencia": int(row['Codigo']),
                "codigoMarca": int(row['cod_marca']),
                "codigoTipoVeiculo": int(row['cod_veiculo']),
                "codigoModelo": int(row['codigoModelo']),
                "anoModelo": int(row['codigoAno'].split('-')[0]),
                "codigoTipoCombustivel": int(row['codigoAno'].split('-')[1]),
                "ano": str(row['codigoAno']),
                "tipoConsulta": "tradicional"
            }

            urlConsulta = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarValorComTodosParametros'
            headers = {'Content-Type': 'application/json'}
            response = requests.post(urlConsulta, headers=headers, data=json.dumps(parametros))
            preco = response.json()

            if 'codigo' in preco and preco['codigo'] == '0' and 'erro' in preco and preco['erro'] == 'nadaencontrado':
                print(f"Erro 'nadaencontrado' no registro {index}. Pulando...")
                success = True
                update_current_index(index + 1, "control_HistoricoPrecos")
                continue

            with open('conferenciaHistoricoPrecos.csv', 'a', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow([preco['Valor'], preco['Marca'], preco['Modelo'], preco['AnoModelo'], preco['Combustivel'], preco['CodigoFipe'], preco['MesReferencia'], preco['Autenticacao'], preco['TipoVeiculo'], preco['SiglaCombustivel'], preco['DataConsulta']])

            update_current_index(index + 1, "control_HistoricoPrecos")

            success = True
        except:
            print(f"Erro ao processar o registro {index}")
            
    del row
    gc.collect()

df_historico = pd.read_csv('conferenciaHistoricoPrecos.csv')

In [ ]:
#SOLUÇÃO BARD

In [5]:
import asyncio
import aiohttp
import csv
import os
import pandas as pd
import json
from typing import Dict

async def fetch(session: aiohttp.ClientSession, url: str, headers: Dict[str, str], data: str) -> Dict[str, any]:
    async with session.post(url, headers=headers, data=data) as response:
        return await response.json()

async def fetch_all(rows: pd.DataFrame, url: str, headers: Dict[str, str]) -> None:
    tasks = []
    async with aiohttp.ClientSession() as session:
        for index, row in rows.iterrows():
            parametros = {
                "codigoTabelaReferencia": int(row['Codigo']),
                "codigoMarca": int(row['cod_marca']),
                "codigoTipoVeiculo": int(row['cod_veiculo']),
                "codigoModelo": int(row['codigoModelo']),
                "anoModelo": int(row['codigoAno'].split('-')[0]),
                "codigoTipoCombustivel": int(row['codigoAno'].split('-')[1]),
                "ano": str(row['codigoAno']),
                "tipoConsulta": "tradicional"
            }
            data = json.dumps(parametros)
            task = fetch(session, url, headers, data)
            tasks.append(task)
        responses = await asyncio.gather(*tasks, return_exceptions=True)
        with open('conferenciaHistoricoPrecos.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for response in responses:
                if isinstance(response, Exception):
                    print(f"Erro: {response}")
                elif 'codigo' in response and response['codigo'] == '0' and 'erro' in response and response['erro'] == 'nadaencontrado':
                    print(f"Erro 'nadaencontrado'. Pulando...")
                else:
                    writer.writerow([response['Valor'], response['Marca'], response['Modelo'], response['AnoModelo'], response['Combustivel'], response['CodigoFipe'], response['MesReferencia'], response['Autenticacao'], response['TipoVeiculo'], response['SiglaCombustivel'], response['DataConsulta']])

if __name__ == '__main__':
    start_index = get_current_index('control_HistoricoPrecos')
    urlConsulta = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarValorComTodosParametros'
    headers = {'Content-Type': 'application/json'}
    df_consulta_total = pd.read_csv('conferenciaHistoricoPrecos.csv')
    loop = asyncio.get_event_loop()
    loop.run_until_complete(fetch_all(df_consulta_total.iloc[start_index:], urlConsulta, headers))


RuntimeError: This event loop is already running

In [ ]:
#SOLUÇÃO CHATGPT

In [10]:
import nest_asyncio
nest_asyncio.apply()

async def fetch(sem: asyncio.Semaphore, session: aiohttp.ClientSession, url: str, headers: Dict[str, str], data: str) -> Dict[str, any]:
    async with sem, session.post(url, headers=headers, data=data) as response:
        return await response.json()

async def fetch_all(rows: pd.DataFrame, url: str, headers: Dict[str, str]) -> None:
    tasks = []
    sem = asyncio.Semaphore(30)  # Limita o número de tarefas simultâneas a 100.
    async with aiohttp.ClientSession() as session:
        for index, row in rows.iterrows():
            parametros = {
                "codigoTabelaReferencia": int(row['Codigo']),
                "codigoMarca": int(row['cod_marca']),
                "codigoTipoVeiculo": int(row['cod_veiculo']),
                "codigoModelo": int(row['codigoModelo']),
                "anoModelo": int(row['codigoAno'].split('-')[0]),
                "codigoTipoCombustivel": int(row['codigoAno'].split('-')[1]),
                "ano": str(row['codigoAno']),
                "tipoConsulta": "tradicional"
            }
            data = json.dumps(parametros)
            task = asyncio.ensure_future(fetch(sem, session, url, headers, data))
            tasks.append(task)
        responses = await asyncio.gather(*tasks, return_exceptions=True)
        with open('conferenciaHistoricoPrecos.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for response in responses:
                if isinstance(response, Exception):
                    print(f"Erro: {response}")
                elif 'codigo' in response and response['codigo'] == '0' and 'erro' in response and response['erro'] == 'nadaencontrado':
                    print(f"Erro 'nadaencontrado'. Pulando...")
                else:
                    writer.writerow([response['Valor'], response['Marca'], response['Modelo'], response['AnoModelo'], response['Combustivel'], response['CodigoFipe'], response['MesReferencia'], response['Autenticacao'], response['TipoVeiculo'], response['SiglaCombustivel'], response['DataConsulta']])

if __name__ == '__main__':
    start_index = get_current_index('control_HistoricoPrecos')
    urlConsulta = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarValorComTodosParametros'
    headers = {'Content-Type': 'application/json'}
    df_consulta_total = pd.read_csv('conferenciaHistoricoPrecos.csv')
    asyncio.ensure_future(fetch_all(df_consulta_total.iloc[start_index:], urlConsulta, headers))